In [1]:
def args_to_str(args:str, var:dict={'SAVED_EMBEDDINGS_DIR':"/vol/bitbucket/jq619/adaptor-thesis/saved_embeddings"}):
    for k, v in var.items():
        args = args.replace('$'+k, v)
    args = args.split(' ')[2:]
    args = [f"\"{arg}\"" for arg in args]
    args = ", \n".join(args)
    print(args)

In [2]:
import os,sys
sys.path.insert(0,"..")
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import sys
import torch.nn.functional as F
import tqdm
import sklearn, sklearn.metrics
import pandas as pd
import torchxrayvision as xrv

In [16]:
model = xrv.autoencoders.ResNetAE(weights="101-elastic")

In [31]:
x = torch.ones(1, 1, 224, 224)
x = model.conv1(x)
x = model.bn1(x)
x = model.relu(x)
x.shape
x = model.maxpool(x)
x.shape
x = model.layer1(x)
x.shape
x = model.layer2(x)
x.shape
# x = model.layer3(x)
# x.shape
# # x = model.layer4(x)
# x.shape

torch.Size([1, 512, 28, 28])

In [8]:
from segmentation_models_pytorch.encoders._base import EncoderMixin
import segmentation_models_pytorch as smp
import torch.nn as nn
import torch
import torchxrayvision as xrv

In [39]:
class ResNetAE101Encoder(nn.Module, EncoderMixin):
    def __init__(self, **kwargs):
        super().__init__()
        self._out_channels = [1, 64, 256, 512, 1024]
        self._depth = 4
        self._in_channels = 1
        self.encoder = xrv.autoencoders.ResNetAE(weights="101-elastic")
    
    def forward(self, x):
        feat0 = x.clone()
        x = self.encoder.conv1(x)
        x = self.encoder.bn1(x)
        feat1 = self.encoder.relu(x)
        x = self.encoder.maxpool(feat1)
        
        feat2 = self.encoder.layer1(x)
        feat3 = self.encoder.layer2(feat2)
        feat4 = self.encoder.layer3(feat3)
        return [feat0, feat1, feat2, feat3, feat4]
        

In [40]:
smp.encoders.encoders["resnet-ae-101"] = {
    "encoder": ResNetAE101Encoder,
    "params":{}, 
    "pretrained_settings":{}, 
}
encoder = ResNetAE101Encoder()
seg_model = smp.Unet(encoder_name="resnet-ae-101", encoder_weights=None, activation=None)

In [41]:
x = torch.ones(1, 1, 224, 224)
features = encoder(x)
[f.shape for f in features]

[torch.Size([1, 1, 224, 224]),
 torch.Size([1, 64, 112, 112]),
 torch.Size([1, 256, 56, 56]),
 torch.Size([1, 512, 28, 28]),
 torch.Size([1, 1024, 14, 14])]

In [ ]:
deg_model

In [49]:
seg_model.decoder(encoder(x)).shape

torch.Size([1, 32, 224, 224])

In [50]:
seg_model.decoder

UnetDecoder(
  (center): Identity()
  (blocks): ModuleList(
    (0): DecoderBlock(
      (conv1): Conv2dReLU(
        (0): Conv2d(1536, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (attention1): Attention(
        (attention): Identity()
      )
      (conv2): Conv2dReLU(
        (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (attention2): Attention(
        (attention): Identity()
      )
    )
    (1): DecoderBlock(
      (conv1): Conv2dReLU(
        (0): Conv2d(512, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inpla

In [76]:
x = torch.ones((1, 1, 224, 224))
x = self.conv1(x)
x = self.bn1(x)
x = self.relu(x)
x = self.maxpool(x)
x.shape
x = self.layer1(x)
x.shape
x = self.layer2(x)
x.shape
x = self.layer3(x)
x.shape
x = self.layer4(x)
x.shape

NameError: name 'self' is not defined

In [ ]:
self.layer4

Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(1024, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(6, 6), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
    (downsample): Sequential(
      (0): Conv2d(1024, 512, kernel_size=(1, 1), stride=(6, 6), bias=False)
      (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (1): Bottleneck(
    (conv1): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(128, 128, 

In [ ]:
from mgca.models.ssl_segmenter import SSLSegmenter

segmenter = SSLSegmenter(seg_model=self)

NameError: name 'self' is not defined

In [ ]:
from torchvision.models import resnet101
from torch.nn import Identity

x = torch.ones((1, 3, 224, 224))
model = resnet101()

model.avgpool = Identity()
model.fc = Identity()
# model.layer4[-1] = Identity()
model(x).shape

torch.Size([1, 100352])

In [ ]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
import torch
import torch.nn as nn

model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')

Using cache found in /homes/jq619/.cache/torch/hub/facebookresearch_dinov2_main


In [ ]:
img = torch.rand(1, 3, 224, 224)
output = model(img, is_training=True)
local = output['x_norm_patchtokens']
cls = output['x_norm_clstoken'].unsqueeze(1)
t = torch.concat([cls, local], dim=1)
t.shape

torch.Size([1, 257, 384])

In [ ]:
torch.manual_seed(1117)
img = torch.rand((1, 3, 224, 224))
model.norm = nn.Identity()
output = model(img)
output.shape

torch.Size([1, 384])

In [ ]:
import torchvision
import torch
vit = torchvision.models.vit_b_16()
torch.manual_seed(1117)
img = torch.rand((1, 3, 224, 224))
img = vit.conv_proj(img)
img.shape
img = vit.encoder.dropout(img)
img.shape
img = img.flatten(start_dim=2).permute(0, 2, 1)
# img = vit.encoder.layers[0](img)
# img.shape
# img = vit.encoder.layers[1](img)
# img.shape
img = vit.encoder.layers(img)
img.shape

torch.Size([1, 196, 768])

In [ ]:
torch.manual_seed(1117)
x = torch.rand((1, 3, 224, 224))
x = vit._process_input(x)
n = x.shape[0]

# Expand the class token to the full batch
batch_class_token = vit.class_token.expand(n, -1, -1)
x = torch.cat([batch_class_token, x], dim=1)

x = vit.encoder(x)

x.shape

torch.Size([1, 197, 768])

In [ ]:
vit

VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a